# Implementing the actual replicator dynamics in Franke and Correia 2018

https://github.com/josepedrocorreia/vagueness-games/blob/master/newCode/vagueness-games.py#L291

In [39]:
import torch

In [40]:
def normalize_rows(mat: torch.Tensor):
    """Normalize each row of 2D array / tensor to sum to 1.0."""
    return torch.nan_to_num(mat / mat.sum(1, keepdim=True))

In [41]:
# For simplicity, let's make things like michael's example.
sender_matrix = torch.tensor(
    [
        # [1., 0.,],
        [0.5, 0.5,],
        [1., 0.,],
        [0., 1.,],
        [0., 1.,],
    ]
)
receiver_matrix = torch.tensor(
    [
    # [1., 0., 0., 0.,],
    [0.25, 0.25, 0.25, 0.25],
    [0., 0., 1., 0.,],
    ]
)

utility_matrix = torch.eye(4)
confusion_matrix = torch.eye(4)
prior = torch.tensor([.4, .4, .1, .1,])
num_states = 4
num_signals = 2

In [42]:
P = sender_matrix  # `[states, signals]`
Q = receiver_matrix  # `[signals, states]`
U = utility_matrix  # `[states, states]`
C = confusion_matrix  # `[states, states]`, compare self.game.meaning_dists
p = prior  # `[states,]`

In [43]:
O = normalize_rows(p * C) # prob actual state (col) given observed state (row)
Sigma = C @ P # prob signal (col) given actual state (row)
O_sender = O @ Sigma # P_o(w|m_o) prob of seeing agent j send signal given agent i sees state m_o
Rho = Q @ C # prob receiver chooses state (column) given signal (row)
U_sender = torch.tensor([
  [
    torch.sum(
        torch.tensor([
            O[to, ta] * Rho[m, tr] * U[ta, tr] 
            for ta in range(num_states) for tr in range(num_states)
        ])
    )
  for m in range(num_signals)
  ]
  for to in range(num_states)]
)
P = normalize_rows(O_sender * U_sender)
P

tensor([[0.5000, 0.5000],
        [1.0000, 0.0000],
        [0.0000, 1.0000],
        [0.0000, 1.0000]])


tensor([[1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 0.]])